In [3]:
!pip install pytest-mock

In [4]:
!pip install pytest pytest-asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.3/342.3 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: pytest
    Found existing installation: pytest 7.4.4
    Uninstalling pytest-7.4.4:
      Successfully uninstalled pytest-7.4.4


In [5]:
%%writefile order_system.py
import asyncio
import threading
from datetime import datetime
from enum import Enum
from typing import List, Optional

class AsyncEventLoop:
    def __init__(self):
        self.loop = asyncio.new_event_loop()
        self.thread = threading.Thread(target=self.run_loop, daemon=True)
        self.thread.start()
        self._event = threading.Event()

    def run_loop(self):
        asyncio.set_event_loop(self.loop)
        self.loop.run_forever()

    def stop(self):
        self.loop.call_soon_threadsafe(self.loop.stop)

    def run_until_complete(self, coro):
        return asyncio.run_coroutine_threadsafe(coro, self.loop).result()

    async def wait(self):
        while not self._event.is_set():
            await asyncio.sleep(0.1)

    def set_event(self):
        self._event.set()

    def clear_event(self):
        self._event.clear()


# Mock classes

class OrderStatus(Enum):
    NONE = "None"
    NEW = "New"
    CANCELED = "Canceled"
    FILLED = "Filled"


class OrderDirection(Enum):
    BUY = "Buy"
    SELL = "Sell"
    HOLD = "Hold"


class OrderType(Enum):
    MARKET = "Market"
    LIMIT = "Limit"


class Symbol:
    def __init__(self, symbol: str):
        self.symbol = symbol

    def __repr__(self):
        return f"Symbol({self.symbol})"


#  Main Order Class.

class Order:
    _incremental_id = 0
    _id_lock = asyncio.Lock()

    def __init__(self, symbol: Symbol, quantity: float, order_time: datetime,event_manager: AsyncEventLoop, price: float = 0.0, tag: str = ''):
        self._id = None
        self.symbol = symbol
        self.quantity = quantity
        self.price = price
        self.price_currency = "USD"
        self.time = order_time
        self.status = OrderStatus.NONE
        self.tag = tag
        self.broker_id: List[str] = []
        self.contingent_id = None
        self.last_fill_time = None
        self.last_update_time = None
        self.canceled_time = None

        self.order_event = event_manager

    @property
    async def id(self):
        if self._id is None:
            async with self._id_lock:
                if self._id is None:
                    self._id = Order._incremental_id
                    Order._incremental_id += 1
        return self._id

    @property
    def direction(self):
        if self.quantity > 0:
            return OrderDirection.BUY
        elif self.quantity < 0:
            return OrderDirection.SELL
        else:
            return OrderDirection.HOLD

    @property
    def absolute_quantity(self):
        return abs(self.quantity)

    @property
    def is_marketable(self):

        return self.price == 0

    async def apply_update(self, quantity: Optional[float] = None, tag: Optional[str] = None):
        """Simulate an update to the order asynchronously."""
        if quantity is not None:
            self.quantity = quantity
        if tag is not None:
            self.tag = tag
        self.last_update_time = datetime.utcnow()

        await asyncio.sleep(0.1)
        self.order_event.set_event()
        return 'update_applied'

    async def cancel(self):
        """Cancel the order asynchronously."""
        self.status = OrderStatus.CANCELED
        self.canceled_time = datetime.utcnow()


        await asyncio.sleep(0.1)
        self.order_event.clear_event()
        return 'order_canceled'

    def __str__(self):
        return f"Order(id={self.id}, symbol={self.symbol}, quantity={self.quantity}, price={self.price}, status={self.status})"

Writing order_system.py


In [6]:
%%writefile test_order_system.py
import pytest
import asyncio
from datetime import datetime
from unittest.mock import patch
from order_system import Order, Symbol, OrderStatus, OrderDirection, AsyncEventLoop




@pytest.fixture
def symbol():
    return Symbol("AAPL")

@pytest.fixture
def order():
    
    event_loop = AsyncEventLoop()
    with patch('order_system.datetime') as mock_datetime:
        mock_datetime.utcnow.return_value = datetime(2024, 1, 1, 12, 0, 0)
        symbol_instance = Symbol("AAPL")
        order_instance = Order(symbol=symbol_instance, quantity=10.0, order_time=mock_datetime.utcnow(), event_manager=event_loop)
        return order_instance

@pytest.mark.asyncio
async def test_order_initialization(order):
    assert order.symbol.symbol == "AAPL"
    assert order.quantity == 10.0
    assert order.status == OrderStatus.NONE
    assert order.time == datetime(2024, 1, 1, 12, 0, 0)
@pytest.mark.asyncio
async def test_order_apply_update(order):
    
    with patch('order_system.datetime') as mock_datetime:
        mock_datetime.utcnow.return_value = datetime(2024, 1, 1, 12, 0, 0)
        await order.apply_update(quantity=15.0, tag="Test update")
        assert order.quantity == 15.0
        assert order.tag == "Test update"
        assert order.last_update_time == datetime(2024, 1, 1, 12, 0, 0)

@pytest.mark.asyncio
async def test_order_cancel(order):
    
    with patch('order_system.datetime') as mock_datetime:
        mock_datetime.utcnow.return_value = datetime(2024, 1, 1, 12, 0, 0)
        await order.cancel()
        assert order.status == OrderStatus.CANCELED
        assert order.canceled_time == datetime(2024, 1, 1, 12, 0, 0)

@pytest.mark.asyncio
async def test_order_direction(order):
    assert order.direction == OrderDirection.BUY

@pytest.mark.asyncio
async def test_order_is_marketable(order):
    order.price = 0.0
    assert order.is_marketable is True
    order.price = 50.0
    assert order.is_marketable is False

Overwriting test_order_system.py


In [7]:
!pytest test_order_system.py --asyncio-mode=auto

/usr/local/lib/python3.10/dist-packages/pytest_asyncio/plugin.py:208: PytestDeprecationWarning: The configuration option "asyncio_default_fixture_loop_scope" is unset.
The event loop scope for asynchronous fixtures will default to the fixture caching scope. Future versions of pytest-asyncio will default the loop scope for asynchronous fixtures to function scope. Set the default fixture loop scope explicitly in order to avoid unexpected behavior in the future. Valid fixture loop scopes are: "function", "class", "module", "package", "session"

  warnings.warn(PytestDeprecationWarning(_DEFAULT_FIXTURE_LOOP_SCOPE_UNSET))
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.3, pluggy-1.5.0
rootdir: /content
plugins: asyncio-0.24.0, mock-3.14.0, anyio-3.7.1, typeguard-4.3.0
asyncio: mode=auto, default_loop_scope=None
collected 5 items                                                                   